In [ ]:
# import syft as sy

# node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True)

In [ ]:
# !pip install veilid==0.2.5

In [1]:
# third party
import veilid

In [2]:
host = "localhost"
port = 5960

In [3]:
async def noop_callback(*args, **kwargs):
    # print("got callback", args, kwargs)
    return

In [4]:
conn = await veilid.json_api_connect(host, port, noop_callback)

In [5]:
# stdlib

In [ ]:
# crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)
# async with crypto_system:
#     print("keygen:Generating a keypair")
#     my_keypair = await crypto_system.generate_key_pair()
#     print(f"keygen:Got {my_keypair=}")

# # await config.store_self_key(conn, my_keypair)

In [6]:
key_pair = "FQkHoYMuJSWhNl3bi2avFstA19QULCiQJtxx_hr2Shg:FrvVQraUvOvv8w0gcgCI_pjyRsXxzlvPjDUmB2_J3MM"

In [7]:
my_keypair = veilid.KeyPair(key_pair)

In [8]:
print("Own keypair:")
print("    Public: ", my_keypair.key())
print("    Private: ", my_keypair.secret())

Own keypair:
    Public:  FQkHoYMuJSWhNl3bi2avFstA19QULCiQJtxx_hr2Shg
    Private:  FrvVQraUvOvv8w0gcgCI_pjyRsXxzlvPjDUmB2_J3MM


In [9]:
their_key = veilid.PublicKey("5MhuM8y3hcL4iAZspNQJJHCQyTVoA-DHVW-so2kmT20")

In [10]:
their_key

'5MhuM8y3hcL4iAZspNQJJHCQyTVoA-DHVW-so2kmT20'

In [11]:
members = [
    veilid.DHTSchemaSMPLMember(my_keypair.key(), 1),
    veilid.DHTSchemaSMPLMember(their_key, 1),
]

In [12]:
# stdlib

In [13]:
key = "VLD0:MeQeEREO1mmzYRm7IZk73V5ia_NaSBVjkxAavpXam5c"

In [14]:
# async def sender(
#     router: veilid.api.RoutingContext,
#     crypto_system: veilid.CryptoSystem,
#     key: veilid.TypedKey,
#     secret: veilid.SharedSecret,
#     send_subkey: veilid.ValueSubkey,
# ):
#     """Read input and write it to the DHT."""

#     async def encrypt(cleartext: str) -> bytes:
#         """Encrypt the message with the shared secret and a random nonce."""

#         print("encrypt:Getting a nonce")
#         nonce = await crypto_system.random_nonce()
#         print(f"encrypt:Received {nonce=}")
#         print(f"encrypt:Encrypting {cleartext=}, {nonce=}, {secret=}")
#         encrypted = await crypto_system.crypt_no_auth(cleartext.encode(), nonce, secret)
#         ciphertext = nonce.to_bytes() + encrypted
#         print(f"encrypt:{ciphertext=}")
#         return ciphertext

#     async def send(cleartext: str):
#         """Write the encrypted version of the text to the DHT."""

#         ciphertext = await encrypt(cleartext)
#         print(f"send:Setting DHT {key=}, {send_subkey=}, to {ciphertext=}")
#         await router.set_dht_value(key, send_subkey, ciphertext)

#     # Prime the pumps. Especially when starting the conversation, this
#     # causes the DHT key to propagate to the network.
#     await send("Hello from the world!")

In [26]:
NONCE_LENGTH = 24
QUIT = "QUIT"

In [27]:
async def receiver(
    router: veilid.api.RoutingContext,
    crypto_system: veilid.CryptoSystem,
    key: veilid.TypedKey,
    secret: veilid.SharedSecret,
    recv_subkey: veilid.ValueSubkey,
    name: str,
):
    """Wait for new data from the DHT and write it to the screen."""

    async def decrypt(payload: bytes) -> str:
        """Decrypt the payload with the shared secret and the payload's nonce."""

        print(f"decrypt:Decrypting {payload!r}")
        nonce = veilid.Nonce.from_bytes(payload[:NONCE_LENGTH])
        print(f"decrypt:Found {nonce=}")
        ciphertext = payload[NONCE_LENGTH:]
        print(f"decrypt:Decrypting {ciphertext=}, {nonce=}, {secret=}")
        cleartext = await crypto_system.crypt_no_auth(ciphertext, nonce, secret)
        print(f"decrypt:{cleartext=}")
        return cleartext.decode()

    last_seq = -1
    while True:
        # In the real world, don't do this. People may tease you for it.
        # This is meant to be easy to understand for demonstration
        # purposes, not a great pattern. Instead, you'd want to use the
        # callback function to handle events asynchronously.

        # Try to get an updated version of the receiving subkey.
        print(f"receiver:Getting the DHT value of {key=}, {recv_subkey=}")
        resp = await router.get_dht_value(key, recv_subkey, True)
        if resp is None:
            print("receiver:Didn't find a value")
            continue

        # If the other party hasn't sent a newer message, try again.
        if resp.seq == last_seq:
            print(f"receiver:DHT {key=} is still at {resp.seq=}")
            continue

        msg = await decrypt(resp.data)
        if msg == QUIT:
            print("receiver:Received the QUIT signal from the other end.")
            print("Other end closed the chat.")
            return

        print("receiver:Got new {msg=} at DHT {key=}, {resp.seq=}")
        print(f"\n{name}> {msg}")
        last_seq = resp.seq

In [20]:
name = "friendsname"

In [21]:
# router = await (await conn.new_routing_context()).with_default_safety()
# crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)
# async with crypto_system, router:
#     print("start:Getting a DH shared secret")
#     secret = await crypto_system.cached_dh(their_key, my_keypair.secret())
#     print(f"start:Got {secret=}")

#     print("start:Creating a new DHT record")
#     record = await router.create_dht_record(veilid.DHTSchema.smpl(0, members))
#     print(f"New chat key: {record.key}")
#     print("Give that to your friend!")

#     # Close this key first. We'll reopen it for writing with our saved key.
#     print(f"start:Closing the DHT record {record.key=}")
#     await router.close_dht_record(record.key)

#     print(f"start:Reopening the DHT record {record.key=} with {my_keypair=}")
#     await router.open_dht_record(record.key, my_keypair)

#     # The party initiating the chat writes to subkey 0 and reads from subkey 1.
#     send_task = asyncio.create_task(sender(router, crypto_system, record.key, secret, 0))
#     recv_task = asyncio.create_task(receiver(router, crypto_system, record.key, secret, 1, name))

#     try:
#         print("start:Starting the chat")
#         await asyncio.wait([send_task, recv_task], return_when=asyncio.FIRST_COMPLETED)
#     finally:
#         print(f"start:Closing the DHT record {record.key=}")
#         await router.close_dht_record(record.key)
#         print(f"start:Deleting the DHT record {record.key=}")
#         await router.delete_dht_record(record.key)

#     print("start:Cleaning up")
#     recv_task.cancel()
#     send_task.cancel()

In [30]:
router = await (await conn.new_routing_context()).with_default_safety()
crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)

name = "friendsname"

async with crypto_system, router:
    print(f"start:Reopening the DHT record {key=} with {my_keypair=}")
    await router.close_dht_record(key)

start:Reopening the DHT record key='VLD0:MeQeEREO1mmzYRm7IZk73V5ia_NaSBVjkxAavpXam5c' with my_keypair='FQkHoYMuJSWhNl3bi2avFstA19QULCiQJtxx_hr2Shg:FrvVQraUvOvv8w0gcgCI_pjyRsXxzlvPjDUmB2_J3MM'


In [31]:
print("respond:Opening a private routing context")
router = await (await conn.new_routing_context()).with_default_safety()
print("respond:Getting a crypto system")
crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)
async with crypto_system, router:
    print("respond:Getting a DH shared secret")
    secret = await crypto_system.cached_dh(their_key, my_keypair.secret())
    print(f"respond:Got {secret=}")

    print(f"respond:Opening the DHT record {key=} with {my_keypair=}")
    await router.open_dht_record(key, my_keypair)

    # The party responding to the chat writes to subkey 1 and reads from subkey 0.
    # send_task = asyncio.create_task(sender(router, crypto_system, key, secret, 1))
    # recv_task = asyncio.create_task(receiver(router, crypto_system, key, secret, 0, name))
    await receiver(router, crypto_system, key, secret, 0, name)

    # try:
    #     LOG.debug("respond:Starting the chat")
    # await asyncio.wait([send_task, recv_task], return_when=asyncio.FIRST_COMPLETED)
    # finally:
    #     LOG.debug(f"respond:Closing the DHT record {key=}")
    #     await router.close_dht_record(key)
    #     LOG.debug(f"respond:Deleting the DHT record {key=}")
    #     await router.delete_dht_record(key)

    # LOG.debug("respond:Cleaning up")
    # recv_task.cancel()
    # send_task.cancel()

respond:Opening a private routing context
respond:Getting a crypto system
respond:Getting a DH shared secret
respond:Got secret='ZcFA4LxhXis2ZPmxhvFnWHQHG1CROLVJTO0_bGgZqi4'
respond:Opening the DHT record key='VLD0:MeQeEREO1mmzYRm7IZk73V5ia_NaSBVjkxAavpXam5c' with my_keypair='FQkHoYMuJSWhNl3bi2avFstA19QULCiQJtxx_hr2Shg:FrvVQraUvOvv8w0gcgCI_pjyRsXxzlvPjDUmB2_J3MM'
receiver:Getting the DHT value of key='VLD0:MeQeEREO1mmzYRm7IZk73V5ia_NaSBVjkxAavpXam5c', recv_subkey=0
decrypt:Decrypting b'\xe3TL*\xa9\n\xbf\xbb\xa6\x86\xa5\xa7,\xc7:\xf2F\x9a\xa1Z\xa9^\x93\xb4\x03\xad\xa0O\xe7\xe0\xdc\xd0\n\xc0\x97\x95'
decrypt:Found nonce='41RMKqkKv7umhqWnLMc68kaaoVqpXpO0'
decrypt:Decrypting ciphertext=b'\x03\xad\xa0O\xe7\xe0\xdc\xd0\n\xc0\x97\x95', nonce='41RMKqkKv7umhqWnLMc68kaaoVqpXpO0', secret='ZcFA4LxhXis2ZPmxhvFnWHQHG1CROLVJTO0_bGgZqi4'
decrypt:cleartext=b'testtesttest'
receiver:Got new {msg=} at DHT {key=}, {resp.seq=}

friendsname> testtesttest
receiver:Getting the DHT value of key='VLD0:MeQeEREO1m

CancelledError: 

In [28]:
await receiver(router, crypto_system, key, secret, 0, name)

receiver:Getting the DHT value of key='VLD0:MeQeEREO1mmzYRm7IZk73V5ia_NaSBVjkxAavpXam5c', recv_subkey=0


ValueError: Response rc_op does not match request rc_op

In [ ]:
# if await config.load_self_key(conn):
#     print("You already have a keypair.")

#     # print the keypair
#     print(await dump_keystore(host, port))

#     sys.exit(1)

# LOG.debug("keygen:Getting a crypto system")
# crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)
# async with crypto_system:
#     LOG.debug("keygen:Generating a keypair")
#     my_keypair = await crypto_system.generate_key_pair()
#     LOG.debug(f"keygen:Got {my_keypair=}")

# await config.store_self_key(conn, my_keypair)

# print(f"Your new public key is: {my_keypair.key()}")
# print("Share it with your friends!")

In [ ]:
# await config.store_friend_key(conn, name, veilid.PublicKey(pubkey))

In [ ]:
# poetry run chat add-friend MyFriend L0nGkEyStR1ng
# async def store_friend_key(
#     conn: veilid.json_api._JsonVeilidAPI, name: str, pubkey: veilid.PublicKey
# ):
#     """Write a friend's public key to the keystore."""

#     await store_key(conn, f"{FRIEND_PREFIX}{name}", str(pubkey))